In [1]:
from scipy import stats

In [2]:
def ret_pvalue(a, b):
    return stats.ttest_ind(a, b, axis=0, equal_var=True, nan_policy='propagate').pvalue

In [4]:
with open("baselines/labels.txt", "r") as f:
    labels = f.readlines()
labels = [line.strip() for line in labels]

# BLEU-2

In [15]:
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

In [24]:
def cal_b2(path):
    with open("baselines/labels.txt", "r") as f:
        labels = f.readlines()
    labels = [line.strip() for line in labels]
    with open(path, "r", encoding="utf8") as f:
        preds = f.readlines()
    preds = [list(line.strip()) for line in preds]
    bleu2 = []
    for i, pred in enumerate(preds):
        bleu2.append(sentence_bleu([labels[i]], pred, weights=(0, 1, 0, 0)))
    print("BLEU-2:", np.mean(bleu2))
    return bleu2

In [27]:
path1 = "baselines/gpt2_samples_tk10_tp07_tm1o1_rpt1o5_ep3.txt"
path2 = "../src_v7_sent_prmpt_ele-wise-mul/res/renew_lr1e-5_ep5_add_bs96_kl02_tk10_tp07_tm1o1_rpt1o5_1.txt"
path3 = "../src_images2poem/res/lr1e-5_bs256_tk10_tp07_tm1o1_rpt1o5.txt"

b2_gpt2, b2_my, b2_i2p = cal_b2(path1), cal_b2(path2), cal_b2(path3)

BLEU-2: 0.025520931844657133
BLEU-2: 0.026886289254387977
BLEU-2: 0.022160881639356828


In [33]:
ret_pvalue(b2_my, b2_gpt2), ret_pvalue(b2_my, b2_i2p)

(0.2840925975249259, 0.0002847968087085035)

### 计算方差

In [63]:
np.var(b2_my)

0.0004322265775886767

In [106]:
tmp_path = "../src_wo_img_text_v7/res/new_lr1e-5_ep5_add_wo_text_bs96_kl02_tk10_tp07_tm1o1_rpt1o5.txt"
tmp_res = cal_b2(tmp_path)
np.var(tmp_res)

BLEU-2: 0.02617243592073183


0.0004231324265734881

# Distinct-2

In [37]:
from paddlenlp.metrics import Distinct

In [40]:
def cal_dist2(path):
    with open("baselines/labels.txt", "r") as f:
        labels = f.readlines()
    labels = [line.strip() for line in labels]
    with open(path, "r") as f:
        preds = f.readlines()
    preds = [item.strip() for item in preds]
    res = []
    n = 2
    for i in range(0,len(preds),10):
        lines = preds[i:i+10]
        distinct = Distinct(n_size=n)
        for line in lines:
            distinct.add_inst(list(line))
        res.append(distinct.score())
    print(np.mean(res))
    return res

In [42]:
path1 = "baselines/gpt2_samples_tk10_tp07_tm1o1_rpt1o5_ep3.txt"
path2 = "../src_v7_sent_prmpt_ele-wise-mul/res/renew_lr1e-5_ep5_add_bs96_kl02_tk10_tp07_tm1o1_rpt1o5_1.txt"
path3 = "../src_images2poem/res/lr1e-5_bs256_tk10_tp07_tm1o1_rpt1o5.txt"

dist2_gpt2, dist2__my, dist2__i2p = cal_dist2(path1), cal_dist2(path2), cal_dist2(path3)

0.5829944218043763
0.7426630701845105
0.6601902324598453


In [43]:
ret_pvalue(dist2__my, dist2_gpt2), ret_pvalue(dist2__my, dist2__i2p)

(4.567203548098187e-62, 9.356413222842288e-36)

### 计算方差

In [74]:
np.var(dist2__my)

0.0005768928758429675

In [107]:
tmp_path = "../src_wo_img_text_v7/res/new_lr1e-5_ep5_add_wo_text_bs96_kl02_tk10_tp07_tm1o1_rpt1o5.txt"
tmp_res = cal_dist2(tmp_path)
np.var(tmp_res)

0.7527491334181662


0.0005202425955027605

# B.S.

In [31]:
from bert_score import score
import warnings
warnings.filterwarnings("ignore")

In [28]:
def cal_bs(path):
    with open("baselines/labels.txt", "r") as f:
        labels = f.readlines()
    labels = [line.strip() for line in labels]
    with open(path, "r") as f:
        samples = f.readlines()
    samples = [line.strip() for line in samples]
    _, _, F1 = score(samples, labels, lang="zh", verbose=False)
    print(f"System level F1 score: {F1.mean():.3f}")
    return F1.tolist()

In [29]:
path1 = "baselines/gpt2_samples_tk10_tp07_tm1o1_rpt1o5_ep3.txt"
path2 = "../src_v7_sent_prmpt_ele-wise-mul/res/renew_lr1e-5_ep5_add_bs96_kl02_tk10_tp07_tm1o1_rpt1o5_1.txt"
path3 = "../src_images2poem/res/lr1e-5_bs256_tk10_tp07_tm1o1_rpt1o5.txt"

bs_gpt2, bs_my, bs_i2p = cal_bs(path1), cal_bs(path2), cal_bs(path3)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


System level F1 score: 0.576


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


System level F1 score: 0.595


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


System level F1 score: 0.585


In [30]:
ret_pvalue(bs_my, bs_gpt2), ret_pvalue(bs_my, bs_i2p)

(1.340879103980386e-27, 3.1019408168277275e-08)

### 计算方差

In [79]:
np.var(bs_my)

0.0008980903998380314

In [108]:
tmp_path = "../src_wo_img_text_v7/res/new_lr1e-5_ep5_add_wo_text_bs96_kl02_tk10_tp07_tm1o1_rpt1o5.txt"
tmp_res = cal_bs(tmp_path)
np.var(tmp_res)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


System level F1 score: 0.593


0.0008392173745298523

# NNR

In [44]:
def create_ngram_set(input_list, ngram_num):
    if len(input_list) < ngram_num:
        return set()
    if len(input_list) <= ngram_num:
        return {tuple(list(input_list))}
    else:
        return set(zip(*[input_list[i:] for i in range(ngram_num)]))

In [45]:
def cal_new_ngrams_rate(ref_data, thep_data, n):
    res = []
    for i in range(len(ref_data)):
        ref = ref_data[i]
#         lines = [thep_data[i]]
        lines = thep_data[i*10:(i+1)*10]
        ref_unique_ngrams = create_ngram_set(ref, n)
        tmp = []
        for line in lines:
            tmp_unique_ngrams = create_ngram_set(line, n)
            tmp.append(len(tmp_unique_ngrams-ref_unique_ngrams)/len(ref_unique_ngrams|tmp_unique_ngrams))
        res.append(np.mean(tmp))
    print(np.mean(res))
    
    return res

In [53]:
def cal_nnr(path1, path2, n):
    with open(path1, "r") as f:
        data1 = f.readlines()
    data1 = [item.strip() for item in data1]
    with open(path2, "r") as f:
        data2 = f.readlines()
    data2 = [item.strip() for item in data2]
    return cal_new_ngrams_rate(data1, data2, n)

In [59]:
path1 = "../src_v7_sent_prmpt_ele-wise-mul/res/new_lr1e-5_ep5_add_bs96_kl02_tk1_tp0_tm1o1_rpt1o5.txt"
path2 = "../src_v7_sent_prmpt_ele-wise-mul/res/new_lr1e-5_ep5_add_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_disorder.txt"

path1_i2p = "../src_images2poem/res/lr1e-5_bs256_tk1_tp0_tm1o1_rpt1o5.txt"
path2_i2p = "../src_images2poem/res/lr1e-5_bs256_tk1_tp0_tm1o1_rpt1o5_disorder.txt"

nnr1_my, nnr1_i2p = cal_nnr(path1, path2, 1), cal_nnr(path1_i2p, path2_i2p, 1)
nnr2_my, nnr2_i2p = cal_nnr(path1, path2, 2), cal_nnr(path1_i2p, path2_i2p, 2)

0.31534780072626356
0.022549234258275354
0.41097149127061533
0.041066237847202064


In [60]:
ret_pvalue(nnr1_my, nnr1_i2p), ret_pvalue(nnr2_my, nnr2_i2p)

(6.99815124186223e-52, 5.735710939481137e-51)

### 计算方差

In [83]:
np.var(nnr1_my), np.var(nnr2_my)

(0.003206073396215843, 0.0033092310283814068)

In [109]:
tmp_path1 = "../src_wo_img_text_v7/res/new_lr1e-5_ep5_add_wo_text_bs96_kl02_tk1_tp0_tm1o1_rpt1o5.txt"
tmp_path2 = "../src_wo_img_text_v7/res/new_lr1e-5_ep5_add_wo_text_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_disorder.txt"
tmp_res1, tmp_res2 = cal_nnr(tmp_path1, tmp_path2, 1), cal_nnr(tmp_path1, tmp_path2, 2)
np.var(tmp_res1), np.var(tmp_res2)

0.2629367177570771
0.34995524071735656


(0.00488615104832266, 0.006767222387258911)